# Oil Project
### Market Analysis by Countries

In [1]:
import sqlalchemy as sqlalchemy_package
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.declarative import DeferredReflection
from sqlalchemy import *
import pandas as pd
import gmaps
import warnings
import config as creds
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from matplotlib.cm import viridis, plasma
from my_func import GoogleMapClient, GmapsAnalysis

sns.set()
warnings.filterwarnings('ignore')
plt.rc('lines', linewidth=1)
rcParams['figure.figsize'] = 12,6
pd.set_option('display.max_colwidth', 40)
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
# Set up a connection to the postgres server
class postgre_sql():
    def __init__(self, dbase = creds.PGDATABASE, dbschema = 'countries'):
        DATABASE_URL = f"postgres://{creds.PGUSER}:{creds.PGPASSWORD}@{creds.PGHOST}:5433/{dbase}"
        engine = sqlalchemy_package.create_engine(DATABASE_URL,
                connect_args={'options': '-csearch_path={}'.format(dbschema)})
        self.engine = engine
    def auto_map(self):
        # Declare a Base with explicit schema using `automap_base()`
        Base = automap_base(bind=self.engine)
        # Use the Base class to reflect the database tables
        Base.prepare(self.engine, reflect=True)
        return Base
    def read_df(self, table='countries'):
        return pd.read_sql(table, self.engine)

In [3]:
# Print all of the classes mapped to the Base
countries_db = postgre_sql()
base = countries_db.auto_map()
base.classes.keys()

[]

In [4]:
"""
--Top 10 countries by Oil, NGPL and other liquids Production in 2019
SELECT o.country, round(AVG(o.oil_prod),2) AS avg_oil
FROM countries AS c
INNER JOIN oil_production as o
ON c.country_code = o.code
INNER JOIN petrol_production as p
ON c.country_code = p.code
WHERE o.year = 2019
GROUP BY o.country
ORDER BY avg_oil DESC
LIMIT 10
"""

'\n--Top 10 countries by Oil, NGPL and other liquids Production in 2019\nSELECT o.country, round(AVG(o.oil_prod),2) AS avg_oil\nFROM countries AS c\nINNER JOIN oil_production as o\nON c.country_code = o.code\nINNER JOIN petrol_production as p\nON c.country_code = p.code\nWHERE o.year = 2019\nGROUP BY o.country\nORDER BY avg_oil DESC\nLIMIT 10\n'

In [13]:
sql_product = base.classes.oil_production
countries = base.classes.countries
# Create a session
session = Session(countries_db.engine)
# Use the session to query countries table and display data for 2019
stmt = session.query(sql_product.country, func.avg(sql_product.oil_prod).label('production')).\
                    filter(sql_product.year == 2019).group_by(sql_product.country).\
                    order_by(desc('production')).statement
data = pd.read_sql_query(stmt, session.bind)
data.head(10)

,country,production
0,United States,"18,388.33"
1,Saudi Arabia,"11,789.92"
2,Russia,"11,457.83"
3,Canada,"5,418.92"
4,Iraq,"4,755.00"
5,China,"4,533.83"
6,United Arab Emirates,"3,996.17"
7,Brazil,"3,595.67"
8,Iran,"3,214.00"
9,Kuwait,"2,940.83"


In [23]:
sql_product = base.classes.oil_production
countries = base.classes.countries
# Create a session
session = Session(countries_db.engine)
# Use the session to query countries table and display data for 2019
stmt = session.query(sql_product.country, countries.country_code, func.avg(sql_product.oil_prod).label('production')).\
                    join(countries, sql_product.code == countries.country_code).\
                    filter(sql_product.year == 2019).group_by(sql_product.country, countries.country_code).\
                    order_by(desc('production')).statement
data = pd.read_sql_query(stmt, session.bind)
data.head()

,country,country_code,production
0,United States,USA,"18,388.33"
1,Saudi Arabia,SAU,"11,789.92"
2,Russia,RUS,"11,457.83"
3,Canada,CAN,"5,418.92"
4,Iraq,IRQ,"4,755.00"


In [24]:
data.to_csv('../outputs/product_2019.csv')

In [7]:
oil_map = GmapsAnalysis(data)
oil_map.color_map(cmap_color = plasma)
oil_map.fig

Figure(layout=FigureLayout(height='420px'))

![oil_map](images/oil_prod_2019.png)